In [2]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import StandardScaler
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import ta
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier


In [3]:

file_path = 'EA_stock_data.csv'
ea_df = pd.read_csv(file_path)
print(ea_df.head())

         Date       Open       High        Low      Close  Adj Close   Volume
0  2019-03-01  96.830002  97.940002  95.309998  97.410004  95.533249  4443600
1  2019-03-04  98.309998  99.430000  95.570000  97.290001  95.415581  7187200
2  2019-03-05  96.260002  97.059998  95.150002  95.720001  93.875832  6041900
3  2019-03-06  95.320000  96.379997  94.129997  94.769997  92.944115  3987500
4  2019-03-07  95.000000  99.559998  94.470001  99.360001  97.445694  8866800


RF MODEL WITHout TA


In [6]:
ea_df = pd.read_csv(file_path)
ea_df.dropna(inplace=True)
ea_df['Target'] = (ea_df['Adj Close'].shift(-80) > ea_df['Adj Close']).astype(int)
ea_df['Target'] = ea_df['Target'].astype('category')
ea_df['Smoothed_Close'] = ea_df['Close'].ewm(alpha=0.3).mean()
features = ['Open',  'Low', 'Close', 'Volume','Smoothed_Close']
X = ea_df[features]
ea_df.dropna(inplace=True)


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y = ea_df['Target'].values


# Assuming 'Target' is your target variable
y = ea_df['Target'].values


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42,shuffle=False)

# Train a Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=243, random_state=42,max_depth= 5)
rf_classifier.fit(X_train, y_train)

# Predict on the test set
predictions = rf_classifier.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions, zero_division=0))



Accuracy: 0.8492063492063492
              precision    recall  f1-score   support

           0       0.91      0.84      0.87       152
           1       0.78      0.87      0.82       100

    accuracy                           0.85       252
   macro avg       0.84      0.85      0.85       252
weighted avg       0.86      0.85      0.85       252



In [32]:
from sklearn.model_selection import TimeSeriesSplit


tscv = TimeSeriesSplit(n_splits=10)

# Initialize the SVM model
rf = RandomForestClassifier(n_estimators=243, random_state=42,max_depth= 20)

# List to store scores for each fold
accuracy_scores = []

# Perform the time series cross-validation
for train_index, test_index in tscv.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit the model
    rf.fit(X_train, y_train)
    
    # Make predictions
    y_pred = rf.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
    
    # Print the classification report for each fold
    print(classification_report(y_test, y_pred, zero_division=0))

# Average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(f'Average Accuracy across folds: {average_accuracy}')

              precision    recall  f1-score   support

           0       0.15      1.00      0.26        16
           1       1.00      0.07      0.14        96

    accuracy                           0.21       112
   macro avg       0.58      0.54      0.20       112
weighted avg       0.88      0.21      0.15       112

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        32
           1       0.66      0.79      0.72        80

    accuracy                           0.56       112
   macro avg       0.33      0.39      0.36       112
weighted avg       0.47      0.56      0.51       112

              precision    recall  f1-score   support

           0       0.29      1.00      0.44        22
           1       1.00      0.39      0.56        90

    accuracy                           0.51       112
   macro avg       0.64      0.69      0.50       112
weighted avg       0.86      0.51      0.54       112

              preci

RF MODEL WITH TA

In [3]:

file_path = 'EA_stock_data.csv'
# Assuming ea_df is your DataFrame loaded with stock data including 'Open', 'High', 'Low', 'Close', 'Volume'
ea_df = pd.read_csv(file_path)
# Calculate Moving Average
ea_df['SMA_10'] = ea_df['Close'].rolling(window=20).mean()

# Calculate MACD
ea_df['MACD'] = ta.trend.MACD(ea_df['Close']).macd()

# Calculate ADX
ea_df['ADX'] = ta.trend.ADXIndicator(ea_df['High'], ea_df['Low'], ea_df['Close'], window=14).adx()
ea_df['Smoothed_Close'] = ea_df['Close'].ewm(alpha=0.3).mean()
# Drop any rows with NaN values that were introduced by the indicator calculations
ea_df.dropna(inplace=True)

# Define the target variable: 1 if the price goes up next week, 0 otherwise
ea_df['Target'] = (ea_df['Adj Close'].shift(-80) > ea_df['Adj Close']).astype(int)
ea_df['Target'] = ea_df['Target'].astype('category')

# Update features list to include the new technical indicators
features = ['Open', 'Low', 'Close', 'Volume','SMA_10', 'MACD','Smoothed_Close']
X = ea_df[features]

# Standardize the features

y = ea_df['Target'].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)

# Initialize and train the SVM model
rf_classifier = RandomForestClassifier(n_estimators=476, random_state=42,max_depth= 20) 
rf_classifier.fit(X_train, y_train)

# Make predictions and evaluate the model
predictions = rf_classifier.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions, zero_division=0))

Accuracy: 0.8825910931174089
              precision    recall  f1-score   support

           0       0.88      0.93      0.91       152
           1       0.88      0.80      0.84        95

    accuracy                           0.88       247
   macro avg       0.88      0.87      0.87       247
weighted avg       0.88      0.88      0.88       247



cv for rf model with ta

In [6]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=10)

# Initialize the SVM model
rf = RandomForestClassifier(n_estimators=476, random_state=42,max_depth= 20)

# List to store scores for each fold
accuracy_scores = []

# Perform the time series cross-validation
for train_index, test_index in tscv.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit the model
    rf.fit(X_train, y_train)
    
    # Make predictions
    y_pred = rf.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
    
    # Print the classification report for each fold
    print(classification_report(y_test, y_pred, zero_division=0))

# Average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(f'Average Accuracy across folds: {average_accuracy}')

              precision    recall  f1-score   support

           0       0.15      1.00      0.26        16
           1       1.00      0.07      0.14        96

    accuracy                           0.21       112
   macro avg       0.58      0.54      0.20       112
weighted avg       0.88      0.21      0.15       112

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        32
           1       0.66      0.79      0.72        80

    accuracy                           0.56       112
   macro avg       0.33      0.39      0.36       112
weighted avg       0.47      0.56      0.51       112

              precision    recall  f1-score   support

           0       0.29      1.00      0.44        22
           1       1.00      0.39      0.56        90

    accuracy                           0.51       112
   macro avg       0.64      0.69      0.50       112
weighted avg       0.86      0.51      0.54       112

              preci

rf with ta and ada boost

In [35]:
file_path = 'EA_stock_data.csv'
import ta  # Import the technical analysis library
ea_df = pd.read_csv(file_path)

# Calculate Moving Average
ea_df['SMA_10'] = ea_df['Close'].rolling(window=20).mean()

# Calculate MACD
ea_df['MACD'] = ta.trend.MACD(ea_df['Close']).macd()

# Calculate ADX
ea_df['ADX'] = ta.trend.ADXIndicator(ea_df['High'], ea_df['Low'], ea_df['Close'], window=14).adx()
ea_df['Smoothed_Close'] = ea_df['Close'].ewm(alpha=0.3).mean()
# Drop any rows with NaN values that were introduced by the indicator calculations
ea_df.dropna(inplace=True)

# Define the target variable: 1 if the price goes up next week, 0 otherwise
ea_df['Target'] = (ea_df['Adj Close'].shift(-80) > ea_df['Adj Close']).astype(int)
ea_df['Target'] = ea_df['Target'].astype('category')

# Update features list to include the new technical indicators
features = ['Open', 'Low', 'Close', 'Volume','SMA_10', 'MACD','Smoothed_Close']
X = ea_df[features]

# Standardize the features
y = ea_df['Target'].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)

# Initialize and train the AdaBoost model with RandomForest as the base estimator
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)
base_rf_classifier = RandomForestClassifier(n_estimators=476, random_state=42,max_depth= 20)  # Base estimator
ada_boost_classifier = AdaBoostClassifier(
    base_estimator=base_rf_classifier,random_state=42,n_estimators=92, learning_rate=0.4232124682782118
    
)
ada_boost_classifier.fit(X_train, y_train)

# Make predictions and evaluate the model
predictions = ada_boost_classifier.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions, zero_division=0))
 

Accuracy: 0.8906882591093117
              precision    recall  f1-score   support

           0       0.89      0.94      0.91       152
           1       0.90      0.81      0.85        95

    accuracy                           0.89       247
   macro avg       0.89      0.88      0.88       247
weighted avg       0.89      0.89      0.89       247



In [34]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import ta  # Make sure you have the TA-Lib installed

# Load your data
file_path = 'EA_stock_data.csv'
ea_df = pd.read_csv(file_path)

# Your preprocessing code comes here
ea_df['SMA_10'] = ea_df['Close'].rolling(window=10).mean()

# Calculate MACD
ea_df['MACD'] = ta.trend.MACD(ea_df['Close']).macd()

# Calculate ADX
ea_df['ADX'] = ta.trend.ADXIndicator(ea_df['High'], ea_df['Low'], ea_df['Close'], window=14).adx()
ea_df['Smoothed_Close'] = ea_df['Close'].ewm(alpha=0.3).mean()
# Drop any rows with NaN values that were introduced by the indicator calculations
ea_df.dropna(inplace=True)
ea_df['Target'] = (ea_df['Adj Close'].shift(-80) > ea_df['Adj Close']).astype(int)
ea_df['Target'] = ea_df['Target'].astype('category')


# Update features list to include the new technical indicators
features = ['Open', 'Low', 'Close', 'Volume', 'SMA_10', 'MACD', 'Smoothed_Close']
X = ea_df[features]

# Define the target variable and standardize the features
y = ea_df['Target'].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the time series cross-validator
tscv = TimeSeriesSplit(n_splits=10)

# Initialize the RandomForest classifier as the base estimator
base_rf_classifier = RandomForestClassifier(n_estimators=476, random_state=42, max_depth=20)

# Initialize the AdaBoost model with RandomForest as the base estimator
ada_boost_classifier = AdaBoostClassifier(base_estimator=base_rf_classifier, n_estimators=92, learning_rate=0.4232124682782118, random_state=42)

# List to store scores for each fold
accuracy_scores = []

# Perform the time series cross-validation
for train_index, test_index in tscv.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit the AdaBoost model
    ada_boost_classifier.fit(X_train, y_train)
    
    # Make predictions
    predictions = ada_boost_classifier.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, predictions)
    accuracy_scores.append(accuracy)
    
    # Print the classification report for each fold
    print(classification_report(y_test, predictions, zero_division=0))

# Average accuracy across all folds
average_accuracy = np.mean(accuracy_scores)
print(f'Average Accuracy across folds: {average_accuracy}')

              precision    recall  f1-score   support

           0       0.16      0.94      0.28        16
           1       0.95      0.20      0.33        96

    accuracy                           0.30       112
   macro avg       0.56      0.57      0.30       112
weighted avg       0.84      0.30      0.32       112

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        32
           1       0.68      0.85      0.76        80

    accuracy                           0.61       112
   macro avg       0.34      0.42      0.38       112
weighted avg       0.49      0.61      0.54       112

              precision    recall  f1-score   support

           0       0.30      1.00      0.46        22
           1       1.00      0.43      0.60        90

    accuracy                           0.54       112
   macro avg       0.65      0.72      0.53       112
weighted avg       0.86      0.54      0.58       112

              preci

Best parameters for rf without anything

In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split, TimeSeriesSplit
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from hyperopt.pyll.base import scope
import numpy as np
ea_df = pd.read_csv(file_path)

ea_df['SMA_10'] = ea_df['Close'].rolling(window=10).mean()

# Calculate MACD
ea_df['MACD'] = ta.trend.MACD(ea_df['Close']).macd()

# Calculate ADX
ea_df['ADX'] = ta.trend.ADXIndicator(ea_df['High'], ea_df['Low'], ea_df['Close'], window=14).adx()
ea_df['Smoothed_Close'] = ea_df['Close'].ewm(alpha=0.3).mean()
# Drop any rows with NaN values that were introduced by the indicator calculations
ea_df.dropna(inplace=True)

# Define the target variable: 1 if the price goes up next week, 0 otherwise
ea_df['Target'] = (ea_df['Adj Close'].shift(-80) > ea_df['Adj Close']).astype(int)
ea_df['Target'] = ea_df['Target'].astype('category')

# Update features list to include the new technical indicators
features = ['Open',  'Low', 'Close', 'Volume','Smoothed_Close']
X = ea_df[features]

# Standardize the features

y = ea_df['Target'].cat.codes  # Convert categorical data to integer codes
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

space = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 100, 500, 1)),
    'max_depth': scope.int(hp.quniform('max_depth', 5, 50, 1)),  # Corrected this line
    # Other parameters can be added here as needed
}

# Example Objective function (ensure you define X_scaled, y, and include necessary imports)
def objective(params):
    clf = RandomForestClassifier(**params, random_state=42)
    # Placeholder for cross_val_score, replace X_scaled and y with your data and target variables
    score = cross_val_score(clf, X_scaled, y, scoring='accuracy').mean()
    return {'loss': -score, 'status': STATUS_OK}

# Example of running the optimization (make sure to define or import your data)
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

print("Best parameters:", best)

100%|██████████| 100/100 [03:45<00:00,  2.26s/trial, best loss: -0.7359533919225832]
Best parameters: {'max_depth': 5.0, 'n_estimators': 243.0}


Best hyper parameters for rf with ta and no adaboost

In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split, TimeSeriesSplit
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from hyperopt.pyll.base import scope
import numpy as np
ea_df = pd.read_csv(file_path)

ea_df['SMA_10'] = ea_df['Close'].rolling(window=10).mean()

# Calculate MACD
ea_df['MACD'] = ta.trend.MACD(ea_df['Close']).macd()

# Calculate ADX
ea_df['ADX'] = ta.trend.ADXIndicator(ea_df['High'], ea_df['Low'], ea_df['Close'], window=14).adx()
ea_df['Smoothed_Close'] = ea_df['Close'].ewm(alpha=0.3).mean()
# Drop any rows with NaN values that were introduced by the indicator calculations
ea_df.dropna(inplace=True)

# Define the target variable: 1 if the price goes up next week, 0 otherwise
ea_df['Target'] = (ea_df['Adj Close'].shift(-80) > ea_df['Adj Close']).astype(int)
ea_df['Target'] = ea_df['Target'].astype('category')

# Update features list to include the new technical indicators
features = ['Open', 'Low', 'Close', 'Volume','SMA_10', 'MACD','Smoothed_Close']
X = ea_df[features]

# Standardize the features

y = ea_df['Target'].cat.codes  # Convert categorical data to integer codes
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

space = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 100, 500, 1)),
    'max_depth': scope.int(hp.quniform('max_depth', 5, 50, 1)),  # Corrected this line
    # Other parameters can be added here as needed
}

# Example Objective function (ensure you define X_scaled, y, and include necessary imports)
def objective(params):
    clf = RandomForestClassifier(**params, random_state=42)
    # Placeholder for cross_val_score, replace X_scaled and y with your data and target variables
    score = cross_val_score(clf, X_scaled, y, scoring='accuracy').mean()
    return {'loss': -score, 'status': STATUS_OK}

# Example of running the optimization (make sure to define or import your data)
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

print("Best parameters:", best)


100%|██████████| 100/100 [05:36<00:00,  3.37s/trial, best loss: -0.7084230275501134]
Best parameters: {'max_depth': 5.0, 'n_estimators': 476.0}


Hyper Parameters for ta and ada

In [ ]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from hyperopt.pyll.base import scope
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_score
# Ensure you have imported necessary modules and defined your dataset (X_scaled and y)


ea_df = pd.read_csv(file_path)

ea_df['SMA_10'] = ea_df['Close'].rolling(window=10).mean()

# Calculate MACD
ea_df['MACD'] = ta.trend.MACD(ea_df['Close']).macd()

# Calculate ADX
ea_df['ADX'] = ta.trend.ADXIndicator(ea_df['High'], ea_df['Low'], ea_df['Close'], window=14).adx()
ea_df['Smoothed_Close'] = ea_df['Close'].ewm(alpha=0.3).mean()
# Drop any rows with NaN values that were introduced by the indicator calculations
ea_df.dropna(inplace=True)

# Define the target variable: 1 if the price goes up next week, 0 otherwise
ea_df['Target'] = (ea_df['Adj Close'].shift(-80) > ea_df['Adj Close']).astype(int)
ea_df['Target'] = ea_df['Target'].astype('category')

# Update features list to include the new technical indicators
features = ['Open', 'Low', 'Close', 'Volume','SMA_10', 'MACD','Smoothed_Close']
X = ea_df[features]

# Standardize the features

y = ea_df['Target'].cat.codes  # Convert categorical data to integer codes
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
space = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 100, 500, 1)),
    'max_depth': scope.int(hp.quniform('max_depth', 5, 50, 1)),
    'n_estimators_ab': scope.int(hp.quniform('n_estimators_ab', 50, 200, 1)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 1.0,2),
    # If you decide to include the algorithm parameter:
    # 'algorithm': hp.choice('algorithm', ['SAMME', 'SAMME.R']),
}

def objective(params):
    # Create the base estimator with corrected parameters
    rf_base = RandomForestClassifier(
        n_estimators=params['n_estimators'],  # Corrected to match key in `space`
        max_depth=params['max_depth'],        # Corrected to match key in `space`
        random_state=42
    )
    
    # Create the AdaBoost model with the RandomForest base estimator
    clf = AdaBoostClassifier(
        base_estimator=rf_base,
        n_estimators=params['n_estimators_ab'],
        learning_rate=params['learning_rate'],
        random_state=42
        # If including 'algorithm', ensure you map the index to the actual string value:
        # algorithm=['SAMME', 'SAMME.R'][params['algorithm']],
    )
    
    # Use a consistent cross-validation method
    score = cross_val_score(clf, X_scaled, y, scoring='accuracy').mean()
    return {'loss': -score, 'status': STATUS_OK}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=200,
            trials=trials)

print("Best parameters:", best)


100%|██████████| 200/200 [1:55:00<00:00, 34.50s/trial, best loss: -0.6857410881801125]   
Best parameters: {'learning_rate': 0.011098446043865652, 'max_depth': 6.0, 'n_estimators': 323.0, 'n_estimators_ab': 50.0}


In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the data
file_path = 'EA_stock_data.csv'
ea_df = pd.read_csv(file_path)

# Preprocess the data
ea_df.dropna(inplace=True)
ea_df['Target'] = (ea_df['Adj Close'].shift(-80) > ea_df['Adj Close']).astype(int)
ea_df['Target'] = ea_df['Target'].astype('category')
ea_df['Smoothed_Close'] = ea_df['Close'].ewm(alpha=0.3).mean()
ea_df.dropna(inplace=True)

# Define features and target variable
features = ['Open', 'Low', 'Close', 'Volume', 'Smoothed_Close']
X = ea_df[features]
y = ea_df['Target'].values

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Set up the TimeSeriesSplit cross-validator
tscv = TimeSeriesSplit(n_splits=10)

# Initialize the Random Forest model
rf_classifier = RandomForestClassifier(n_estimators=243, max_depth=5, random_state=42)

# List to store scores and create a loop to perform cross-validation
accuracy_scores = []

for train_index, test_index in tscv.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the Random Forest model
    rf_classifier.fit(X_train, y_train)

    # Make predictions on the test set
    predictions = rf_classifier.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, predictions)
    accuracy_scores.append(accuracy)
    print("Fold Accuracy:", accuracy)
    print(classification_report(y_test, predictions, zero_division=0))

# Calculate and print the average accuracy
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print("Average Accuracy across all folds:", average_accuracy)


Fold Accuracy: 0.30701754385964913
              precision    recall  f1-score   support

           0       0.17      1.00      0.29        16
           1       1.00      0.19      0.32        98

    accuracy                           0.31       114
   macro avg       0.58      0.60      0.31       114
weighted avg       0.88      0.31      0.32       114

Fold Accuracy: 0.7719298245614035
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        16
           1       0.85      0.90      0.87        98

    accuracy                           0.77       114
   macro avg       0.42      0.45      0.44       114
weighted avg       0.73      0.77      0.75       114

Fold Accuracy: 0.7280701754385965
              precision    recall  f1-score   support

           0       0.54      1.00      0.70        36
           1       1.00      0.60      0.75        78

    accuracy                           0.73       114
   macro avg       0.77  

In [3]:
import pandas as pd
import ta  # Technical Analysis library for financial indicators
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the data
file_path = 'EA_stock_data.csv'
ea_df = pd.read_csv(file_path)

# Calculate technical indicators
ea_df['SMA_10'] = ea_df['Close'].rolling(window=20).mean()
ea_df['MACD'] = ta.trend.MACD(ea_df['Close']).macd()
ea_df['ADX'] = ta.trend.ADXIndicator(ea_df['High'], ea_df['Low'], ea_df['Close'], window=14).adx()
ea_df['Smoothed_Close'] = ea_df['Close'].ewm(alpha=0.3).mean()

# Drop rows with NaN values that may have been introduced by indicator calculations
ea_df.dropna(inplace=True)

# Define the target variable for future price increase
ea_df['Target'] = (ea_df['Adj Close'].shift(-80) > ea_df['Adj Close']).astype(int)
ea_df['Target'] = ea_df['Target'].astype('category')

# Select features
features = ['Open', 'Low', 'Close', 'Volume', 'SMA_10', 'MACD', 'Smoothed_Close']
X = ea_df[features]
y = ea_df['Target'].values

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Set up the TimeSeriesSplit cross-validator
tscv = TimeSeriesSplit(n_splits=10)

# Initialize the Random Forest model
rf_classifier = RandomForestClassifier(n_estimators=476, max_depth=20, random_state=42)

# List to store scores and create a loop to perform cross-validation
accuracy_scores = []

for train_index, test_index in tscv.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the Random Forest model
    rf_classifier.fit(X_train, y_train)

    # Make predictions on the test set
    predictions = rf_classifier.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, predictions)
    accuracy_scores.append(accuracy)
    print("Fold Accuracy:", accuracy)
    print(classification_report(y_test, predictions, zero_division=0))

# Calculate and print the average accuracy
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print("Average Accuracy across all folds:", average_accuracy)


Fold Accuracy: 0.20535714285714285
              precision    recall  f1-score   support

           0       0.15      1.00      0.26        16
           1       1.00      0.07      0.14        96

    accuracy                           0.21       112
   macro avg       0.58      0.54      0.20       112
weighted avg       0.88      0.21      0.15       112

Fold Accuracy: 0.5625
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        32
           1       0.66      0.79      0.72        80

    accuracy                           0.56       112
   macro avg       0.33      0.39      0.36       112
weighted avg       0.47      0.56      0.51       112

Fold Accuracy: 0.5089285714285714
              precision    recall  f1-score   support

           0       0.29      1.00      0.44        22
           1       1.00      0.39      0.56        90

    accuracy                           0.51       112
   macro avg       0.64      0.69    

In [4]:
import pandas as pd
import ta  # Technical Analysis library
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the data
file_path = 'EA_stock_data.csv'
ea_df = pd.read_csv(file_path)

# Calculate technical indicators
ea_df['SMA_10'] = ea_df['Close'].rolling(window=20).mean()
ea_df['MACD'] = ta.trend.MACD(ea_df['Close']).macd()
ea_df['ADX'] = ta.trend.ADXIndicator(ea_df['High'], ea_df['Low'], ea_df['Close'], window=14).adx()
ea_df['Smoothed_Close'] = ea_df['Close'].ewm(alpha=0.3).mean()

# Drop rows with NaN values introduced by indicator calculations
ea_df.dropna(inplace=True)

# Define the target variable
ea_df['Target'] = (ea_df['Adj Close'].shift(-80) > ea_df['Adj Close']).astype(int)
ea_df['Target'] = ea_df['Target'].astype('category')

# Prepare features
features = ['Open', 'Low', 'Close', 'Volume', 'SMA_10', 'MACD', 'Smoothed_Close']
X = ea_df[features]
y = ea_df['Target'].values

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Set up TimeSeriesSplit for cross-validation
tscv = TimeSeriesSplit(n_splits=10)

# Initialize AdaBoost with RandomForest as base estimator
base_rf_classifier = RandomForestClassifier(n_estimators=476, max_depth=20, random_state=42)
ada_boost_classifier = AdaBoostClassifier(
    base_estimator=base_rf_classifier,
    random_state=42,
    n_estimators=92,
    learning_rate=0.4232124682782118
)

# List to store scores and create a loop to perform cross-validation
accuracy_scores = []

for train_index, test_index in tscv.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train AdaBoost with RandomForest base
    ada_boost_classifier.fit(X_train, y_train)

    # Make predictions
    predictions = ada_boost_classifier.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, predictions)
    accuracy_scores.append(accuracy)
    print("Fold Accuracy:", accuracy)
    print(classification_report(y_test, predictions, zero_division=0))

# Calculate and print the average accuracy
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print("Average Accuracy across all folds:", average_accuracy)


Fold Accuracy: 0.20535714285714285
              precision    recall  f1-score   support

           0       0.15      1.00      0.26        16
           1       1.00      0.07      0.14        96

    accuracy                           0.21       112
   macro avg       0.58      0.54      0.20       112
weighted avg       0.88      0.21      0.15       112

Fold Accuracy: 0.5625
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        32
           1       0.66      0.79      0.72        80

    accuracy                           0.56       112
   macro avg       0.33      0.39      0.36       112
weighted avg       0.47      0.56      0.51       112

Fold Accuracy: 0.44642857142857145
              precision    recall  f1-score   support

           0       0.26      1.00      0.42        22
           1       1.00      0.31      0.47        90

    accuracy                           0.45       112
   macro avg       0.63      0.66   